In [1]:
import sys
import traceback
from decimal import Decimal
import pandas_ta as ta  # noqa: F401

from hummingbot.core.data_type.common import PositionMode, TradeType, OrderType
from hummingbot.data_feed.candles_feed.candles_factory import CandlesConfig
from hummingbot.smart_components.strategy_frameworks.data_types import TripleBarrierConf, OrderLevel
from hummingbot.smart_components.strategy_frameworks.directional_trading import DirectionalTradingBacktestingEngine
from hummingbot.smart_components.utils.config_encoder_decoder import ConfigEncoderDecoder
from optuna import TrialPruned
from hummingbot.smart_components.utils.order_level_builder import OrderLevelBuilder

from quants_lab.controllers.supertrend import SuperTrend, SuperTrendConfig

# To run an optimization with optuna we need to define the objective function that will be executed for each trial

def objective(trial):
    try:
        # Market configuration
        exchange = "binance_perpetual"
        trading_pair = "WLD-USDT"
        interval = "3m"

        # Account configuration
        initial_portfolio_usd = 1000
        order_amount = Decimal("25")
        n_levels = 1
        leverage = 20
        trade_cost = 0.0006

        # Backtest period
        start = "2023-01-01"
        end = "2024-01-02"

        # Triple barrier configuration
        stop_loss = trial.suggest_float('stop_loss', 0.01, 0.02, step=0.01)
        take_profit = trial.suggest_float('take_profit', 0.01, 0.04, step=0.01)
        time_limit = 60 * 60 * 12  # 12 hours
        trailing_stop_activation_price_delta = Decimal("0.008")
        trailing_stop_trailing_delta = Decimal("0.004")

        length = trial.suggest_int('length', 20, 200, step=20)
        multiplier = trial.suggest_float('multiplier', 2.0, 6.0, step=1.0)
        percentage_threshold = trial.suggest_float('percentage_threshold', 0.01, 0.03, step=0.01)

        # Building the order levels
        order_level_builder = OrderLevelBuilder(n_levels=n_levels)
        order_levels = order_level_builder.build_order_levels(
            amounts=order_amount,
            spreads=Decimal("0"),
            triple_barrier_confs=TripleBarrierConf(
                stop_loss=stop_loss, take_profit=take_profit, time_limit=time_limit,
                trailing_stop_activation_price_delta=trailing_stop_activation_price_delta,
                trailing_stop_trailing_delta=trailing_stop_trailing_delta),
        )
        config = SuperTrendConfig(
            exchange=exchange,
            trading_pair=trading_pair,
            strategy_name='supertrend',
            candles_config=[
                CandlesConfig(connector=exchange, trading_pair=trading_pair,
                              interval=interval, max_records=sys.maxsize)
            ],
            order_levels=order_levels,
            leverage=leverage,
            position_mode=PositionMode.HEDGE,
            length=length,
            multiplier=multiplier,
            percentage_threshold=percentage_threshold,

        )
        controller = SuperTrend(config=config)
        engine = DirectionalTradingBacktestingEngine(controller=controller)
        engine.load_controller_data("../../data/candles")
        backtesting_results = engine.run_backtesting(
            initial_portfolio_usd=initial_portfolio_usd,
            trade_cost=trade_cost,
            start=start, end=end)

        strategy_analysis = backtesting_results["results"]
        encoder_decoder = ConfigEncoderDecoder(TradeType, OrderType, PositionMode)

        trial.set_user_attr("net_pnl_quote", strategy_analysis["net_pnl_quote"])
        trial.set_user_attr("net_pnl_pct", strategy_analysis["net_pnl"])
        trial.set_user_attr("max_drawdown_usd", strategy_analysis["max_drawdown_usd"])
        trial.set_user_attr("max_drawdown_pct", strategy_analysis["max_drawdown_pct"])
        trial.set_user_attr("sharpe_ratio", strategy_analysis["sharpe_ratio"])
        trial.set_user_attr("accuracy", strategy_analysis["accuracy"])
        trial.set_user_attr("total_positions", strategy_analysis["total_positions"])
        trial.set_user_attr("profit_factor", strategy_analysis["profit_factor"])
        trial.set_user_attr("duration_in_hours", strategy_analysis["duration_minutes"] / 60)
        trial.set_user_attr("avg_trading_time_in_hours", strategy_analysis["avg_trading_time_minutes"] / 60)
        trial.set_user_attr("win_signals", strategy_analysis["win_signals"])
        trial.set_user_attr("loss_signals", strategy_analysis["loss_signals"])
        trial.set_user_attr("config", encoder_decoder.encode(config.dict()))
        return strategy_analysis["net_pnl"]
    except Exception as e:
        traceback.print_exc()
        raise TrialPruned()


In [2]:
import optuna

# Now let's configure the parameters for the optimization
study_name = "super_trend_optimization_1"
storage= "sqlite:///../../data/backtesting/backtesting_report.db"

study = optuna.create_study(direction="maximize", study_name=study_name,
                            storage=storage,
                            load_if_exists=True # If the study already exists, we load it
                            )

[I 2023-12-07 17:10:50,338] Using an existing study with name 'super_trend_optimization_1' instead of creating a new one.


In [ ]:
# Not let's run the optimization!

n_trials = 200
study.optimize(objective, n_trials=n_trials)

[I 2023-12-07 17:10:56,238] Trial 23 finished with value: 0.38811148004126206 and parameters: {'stop_loss': 0.01, 'take_profit': 0.03, 'length': 100, 'multiplier': 3.0, 'percentage_threshold': 0.01}. Best is trial 13 with value: 0.4226868845214273.
[I 2023-12-07 17:11:00,180] Trial 24 finished with value: 0.38811148004126206 and parameters: {'stop_loss': 0.01, 'take_profit': 0.03, 'length': 100, 'multiplier': 3.0, 'percentage_threshold': 0.01}. Best is trial 13 with value: 0.4226868845214273.
[I 2023-12-07 17:11:05,299] Trial 25 finished with value: 0.2513856255976169 and parameters: {'stop_loss': 0.01, 'take_profit': 0.03, 'length': 100, 'multiplier': 2.0, 'percentage_threshold': 0.01}. Best is trial 13 with value: 0.4226868845214273.
[I 2023-12-07 17:11:09,145] Trial 26 finished with value: 0.2543681375647173 and parameters: {'stop_loss': 0.01, 'take_profit': 0.03, 'length': 140, 'multiplier': 3.0, 'percentage_threshold': 0.01}. Best is trial 13 with value: 0.4226868845214273.
[I 202